In [ ]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers, preprocessing
from keras.constraints import maxnorm
from tensorflow.keras import datasets, layers, models, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [ ]:
# Loading Data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train.npy')
print('Data:', data_train.shape)
print('Labels:', labels_train.shape, '\n')

# Split the data into training and test
X_train, X_test, y_train, y_test = train_test_split(data_train.T, labels_train, test_size=0.2)
print('Training and Test Split')
print('Train data:', X_train.shape)
print('Test data:', X_test.shape)
print('Train labels:', y_train.shape)
print('Test labels:', y_test.shape, '\n')

# Names of the classes
classes = ['Stadium', 'Building/s', 'Traffic Sign', 
           'Forest', 'Flower/s', 'Street', 
           'Classroom', 'Bridge', 'Statue', 'Lake']

# Reshape the data
data_train_reshaped = (data_train / 255.0).reshape(-1,300,300,3)
print('Scaled data:',data_train_reshaped.shape, '\n')

# Function plots a sample
def plot_sample(X, y, index):
    plt.figure(figsize = (30,5))
    plt.imshow(X[:,index].reshape(300,300,3))
    plt.xlabel(classes[int(y[int(index)])-1])

# Adjust the labels to start at 0
new_labels = labels_train - 1
print('New labels:', np.unique(new_labels),'\n')

# One hot encoding 
one_hot_labels = tf.keras.utils.to_categorical(new_labels)
train_one_hot_labels = tf.keras.utils.to_categorical(y_train-1)

print('Size of the one hot labels:',one_hot_labels.shape,'\n')
np.save('one_hot_labels',one_hot_labels)

tf.compat.v1.reset_default_graph()

In [ ]:
def plt_history(hist):
    
    fig,ax = plt.subplots(2,1,figsize=(10,7))
    
    # Plot Accuracy
    ax[0].plot(hist.history['accuracy'], label='Train Accuracy')
    ax[0].plot(hist.history['val_accuracy'], label='Test Accuracy')
    ax[0].set_ylabel('Accuracy')
    ax[0].legend(loc='lower right')
    ax[0].set_title('Accuracy Evaluation')
    
    # Plot Error
    ax[1].plot(hist.history['loss'], label='Train Error')
    ax[1].plot(hist.history['val_loss'], label='Test Error')
    ax[1].set_ylabel('Error')
    ax[1].legend(loc='upper right')
    ax[1].set_title('Error Evaluation')
    
    plt.show()

In [ ]:
from keras.applications.xception import preprocess_input, Xception
xception_model = Xception(weights='imagenet', include_top=False, input_shape=(300,300,3))

In [ ]:
X_train_pp = preprocess_input(X_train.reshape(-1,300,300,3))
data_train_preprocessed = preprocess_input(data_train.T.reshape(-1,300,300,3))

last_layer2 = xception_model.get_layer("block14_sepconv2_bn")

xception_layers = keras.Model(inputs=xception_model.inputs, outputs=last_layer2.output)
xception_model2=Sequential()
xception_model2.add(xception_layers)
xception_model2.add(Flatten())
xception_model2.add(Dense(10,activation='softmax'))

xception_model2.layers[0].trainable=False

xception_model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
xception_model2.summary()

In [ ]:
data_train.T.reshape(-1,300,300,3).shape

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=15)

In [ ]:
xception_model2.layers[0].trainable = True

for layer in xception_model2.layers[0].layers[:-10]:
    layer.trainable = False
for layer in xception_model2.layers[0].layers[-10:]:
    layer.trainable = True

opt = keras.optimizers.Adam(learning_rate=0.0001)
xception_model2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

history = xception_model2.fit(data_train_preprocessed,
                              one_hot_labels, 
                              batch_size=128, 
                              epochs=200,
                              validation_split = 0.2, 
                              callbacks=[callback])
plt_history(history)
xception_model2.summary()

In [ ]:
# Save the model
xception_model2.save('my_model.hdf5')